In [1]:
%load_ext autoreload
%autoreload 2

# Introduction and Objective
## Training with MultiTask Learning

In [2]:
import h5py
import numpy as np
import yaml
import os
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd

In [3]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

In [6]:
from utils import (
    HDF5MultitaskDataset,
    ResizeTransform, 
    MultitaskCollator,
    MultiTaskLandmarkUNetCustom,
    nested_dict_to_easydict,
    Coord2HeatmapTransform,
    CustomToTensor,
)

# load parameters

In [38]:
with open("../../code_configs/params.yaml") as f:
    PARAMS = yaml.safe_load(f)
    PARAMS = nested_dict_to_easydict(PARAMS)

# Load metadata table

In [8]:
metadata_table = pd.read_hdf(
    os.path.join(PARAMS.PRIMARY_DATA_DIRECTORY, PARAMS.TRAIN.METADATA_TABLE_NAME),
    key='df',
)

In [7]:
metadata_table.head()

,source_image_filename,harmonized_id,dataset,dev_set,v_annots_present,f_annots_present,edges_present,split
0,45.jpg,041281ee7fb89f6835a71c309b3b503e3d5a68fc46a608...,dataset_1,NaN,True,False,True,undefined
1,92.jpg,2cfa37a69916c8a45a51bb8beeb04425e07d2a22f694e0...,dataset_1,NaN,True,False,True,undefined
2,43.jpg,7201dc2be0b97f59a7901004d6496bbe84c440530776db...,dataset_1,NaN,True,False,True,undefined
3,7.jpg,2cd4487c03c72d1016ea0a72d1b21eb987878c90ae9eff...,dataset_1,NaN,True,False,True,undefined
4,121.jpg,27624a6eb37bbc8aafabe2075f423d573b189eae6f23fb...,dataset_1,NaN,True,False,True,undefined


# DataLoader for task one: Input Image Reconstruction

In [11]:
# define the task id
task_id = 1

# create the right list of paths
train_file_list = metadata_table.loc[
    (metadata_table['split']=='train') , ['harmonized_id']
].to_numpy().ravel().tolist()

train_file_list = [
    os.path.join(PARAMS.PRIMARY_DATA_DIRECTORY, file_path+'.hdf5') for file_path in train_file_list
]

# instantiate the transforms
my_transforms = transforms.Compose([
    ResizeTransform(tuple(PARAMS.TRAIN.TARGET_IMAGE_SIZE)),
    Coord2HeatmapTransform(
        tuple(PARAMS.TRAIN.TARGET_IMAGE_SIZE),
        PARAMS.TRAIN.GAUSSIAN_COORD2HEATMAP_STD
    ),
    CustomToTensor(),
])

# instantiate the dataset and dataloader objects
train_dataset = HDF5MultitaskDataset(
    file_paths=train_file_list,
    task_id=task_id,
    transforms=my_transforms,
)
collator_task = MultitaskCollator(
    task_id=task_id,
)
dataloader_one = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    collate_fn=collator_task
)

In [12]:
# sanity check dataset and dataloader

# dataset
print("-- Sanity check dataset object!")
dataset_iter = iter(train_dataset)
for batch in dataset_iter:
    print(batch.keys())
    for k, v in batch.items():
        print()
        print(k,)
        print(v.shape)
    break

print()

# data loader
print("-- Sanity check dataloader object!")
for batch_ndx, sample in enumerate(dataloader_one):
    print("batch_ndx ", batch_ndx)
    for k, v in sample.items():
        print()
        print(k,)
        print(v.shape)
    break

-- Sanity check dataset object!
dict_keys(['image'])

image
torch.Size([1, 256, 256])

-- Sanity check dataloader object!
batch_ndx  0

image
torch.Size([4, 1, 256, 256])


# DataLoader for task two: Edge Detection

In [13]:
# define the task id
task_id = 2

# create the right list of paths
train_file_list = metadata_table.loc[
    (metadata_table['split']=='train') & (metadata_table['edges_present']==True), ['harmonized_id']
].to_numpy().ravel().tolist()

train_file_list = [
    os.path.join(PARAMS.PRIMARY_DATA_DIRECTORY, file_path+'.hdf5') for file_path in train_file_list
]

# instantiate the transforms
my_transforms = transforms.Compose([
    ResizeTransform(tuple(PARAMS.TRAIN.TARGET_IMAGE_SIZE)),
    Coord2HeatmapTransform(
        tuple(PARAMS.TRAIN.TARGET_IMAGE_SIZE),
        PARAMS.TRAIN.GAUSSIAN_COORD2HEATMAP_STD
    ),
    CustomToTensor(),
])

# instantiate the dataset and dataloader objects
train_dataset = HDF5MultitaskDataset(
    file_paths=train_file_list,
    task_id=task_id,
    transforms=my_transforms,
)
collator_task = MultitaskCollator(
    task_id=task_id,
)
dataloader_two = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    collate_fn=collator_task
)

In [14]:
# sanity check dataset and dataloader

# dataset
print("-- Sanity check dataset object!")
dataset_iter = iter(train_dataset)
for batch in dataset_iter:
    print(batch.keys())
    for k, v in batch.items():
        print()
        print(k,)
        print(v.shape)
    break

print()

# data loader
print("-- Sanity check dataloader object!")
for batch_ndx, sample in enumerate(dataloader_two):
    print("batch_ndx ", batch_ndx)
    for k, v in sample.items():
        print()
        print(k,)
        print(v.shape)
    break

-- Sanity check dataset object!
dict_keys(['image', 'edges'])

image
torch.Size([1, 256, 256])

edges
torch.Size([1, 256, 256])

-- Sanity check dataloader object!
batch_ndx  0

image
torch.Size([4, 1, 256, 256])

edges
torch.Size([4, 1, 256, 256])


# DataLoader for task three: Vertebral Landmark Detection

In [15]:
# define the task id
task_id = 3

# create the right list of paths
train_file_list = metadata_table.loc[
    (metadata_table['split']=='train') & (metadata_table['v_annots_present']==True), ['harmonized_id']
].to_numpy().ravel().tolist()

train_file_list = [
    os.path.join(PARAMS.PRIMARY_DATA_DIRECTORY, file_path+'.hdf5') for file_path in train_file_list
]

# instantiate the transforms
my_transforms = transforms.Compose([
    ResizeTransform(tuple(PARAMS.TRAIN.TARGET_IMAGE_SIZE)),
    Coord2HeatmapTransform(
        tuple(PARAMS.TRAIN.TARGET_IMAGE_SIZE),
        PARAMS.TRAIN.GAUSSIAN_COORD2HEATMAP_STD
    ),
    CustomToTensor(),
])

# instantiate the dataset and dataloader objects
train_dataset = HDF5MultitaskDataset(
    file_paths=train_file_list,
    task_id=task_id,
    transforms=my_transforms,
)
collator_task = MultitaskCollator(
    task_id=task_id,
)
dataloader_three = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    collate_fn=collator_task
)

In [16]:
# sanity check dataset and dataloader

# dataset
print("-- Sanity check dataset object!")
dataset_iter = iter(train_dataset)
for batch in dataset_iter:
    print(batch.keys())
    for k, v in batch.items():
        print()
        print(k,)
        print(v.shape)
    break

print()

# data loader
print("-- Sanity check dataloader object!")
for batch_ndx, sample in enumerate(dataloader_three):
    print("batch_ndx ", batch_ndx)
    for k, v in sample.items():
        print()
        print(k,)
        print(v.shape)
    break

-- Sanity check dataset object!
dict_keys(['image', 'v_landmarks'])

image
torch.Size([1, 256, 256])

v_landmarks
torch.Size([13, 256, 256])

-- Sanity check dataloader object!
batch_ndx  0

image
torch.Size([4, 1, 256, 256])

v_landmarks
torch.Size([4, 13, 256, 256])


# DataLoader for task four: Facial Landmark Detection

In [17]:
# define the task id
task_id = 4

# create the right list of paths
train_file_list = metadata_table.loc[
    (metadata_table['split']=='train') & (metadata_table['f_annots_present']==True), ['harmonized_id']
].to_numpy().ravel().tolist()

train_file_list = [
    os.path.join(PARAMS.PRIMARY_DATA_DIRECTORY, file_path+'.hdf5') for file_path in train_file_list
]

# instantiate the transforms
my_transforms = transforms.Compose([
    ResizeTransform(tuple(PARAMS.TRAIN.TARGET_IMAGE_SIZE)),
    Coord2HeatmapTransform(
        tuple(PARAMS.TRAIN.TARGET_IMAGE_SIZE),
        PARAMS.TRAIN.GAUSSIAN_COORD2HEATMAP_STD
    ),
    CustomToTensor(),
])

# instantiate the dataset and dataloader objects
train_dataset = HDF5MultitaskDataset(
    file_paths=train_file_list,
    task_id=task_id,
    transforms=my_transforms,
)
collator_task = MultitaskCollator(
    task_id=task_id,
)
dataloader_four = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    collate_fn=collator_task
)

In [18]:
# sanity check dataset and dataloader

# dataset
print("-- Sanity check dataset object!")
dataset_iter = iter(train_dataset)
for batch in dataset_iter:
    print(batch.keys())
    for k, v in batch.items():
        print()
        print(k,)
        print(v.shape)
    break

print()

# data loader
print("-- Sanity check dataloader object!")
for batch_ndx, sample in enumerate(dataloader_four):
    print("batch_ndx ", batch_ndx)
    for k, v in sample.items():
        print()
        print(k,)
        print(v.shape)
    break

-- Sanity check dataset object!
dict_keys(['image', 'f_landmarks'])

image
torch.Size([1, 256, 256])

f_landmarks
torch.Size([19, 256, 256])

-- Sanity check dataloader object!
batch_ndx  0

image
torch.Size([4, 1, 256, 256])

f_landmarks
torch.Size([4, 19, 256, 256])


# Model 

In [34]:
model = MultiTaskLandmarkUNetCustom(
    in_channels=1,
    out_channels1=1,
    out_channels2=1,
    out_channels3=13,
    out_channels4=19,
    enc_chan_multiplier=1,
    dec_chan_multiplier=1,
    backbone_encoder="efficientnet-b4",
    backbone_weights="imagenet",
    freeze_backbone=True,
)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b4-6ed6700e.pth" to /home/samehr/.cache/torch/hub/checkpoints/efficientnet-b4-6ed6700e.pth
100%|██████████| 74.4M/74.4M [00:09<00:00, 8.44MB/s]


In [41]:
model_params = PARAMS.MODEL.PARAMS
model = MultiTaskLandmarkUNetCustom(**model_params)

In [35]:
# count the number of trainable parameters
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total number of trainable parameters: ", num_params)

Total number of trainable parameters:  450266


In [36]:
image = torch.randn(1, 1, 256, 256)
image /= image.max()

In [37]:
out = model(image, task_id=3)
print(out.shape)

torch.Size([1, 13, 256, 256])


# Test Pytorch Lightning

In [44]:
from utils import (
    trainer_v_landmarks_single_task,
)
import pytorch_lightning as pl
import torch

In [47]:
trainer_v_landmarks_single_task()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name      | Type                        | Params
----------------------------------------------------------
0 | model     | MultiTaskLandmarkUNetCustom | 18.0 M
1 | train_mse | MeanSquaredError            | 0     
2 | val_mse   | MeanSquaredError            | 0     
----------------------------------------------------------
450 K     Trainable params
17.5 M    Non-trainable params
18.0 M    Total params
71.992    Total estimated model params size (MB)


Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:10<00:10, 10.43s/it]

KeyError: Caught KeyError in DataLoader worker process 1.
Original Traceback (most recent call last):
  File "/home/samehr/Desktop/cephal/cvmt/.venv/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/samehr/Desktop/cephal/cvmt/.venv/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 54, in fetch
    return self.collate_fn(data)
  File "/home/samehr/Desktop/cephal/cvmt/notebooks/ml/utils.py", line 245, in __call__
    out = self.__supervised_training_collation(batch)
  File "/home/samehr/Desktop/cephal/cvmt/notebooks/ml/utils.py", line 200, in __supervised_training_collation
    batch = [sample for sample in batch if sample[self.label_key] != None]
  File "/home/samehr/Desktop/cephal/cvmt/notebooks/ml/utils.py", line 200, in <listcomp>
    batch = [sample for sample in batch if sample[self.label_key] != None]
KeyError: 'v_landmarks'


In [48]:
from utils import create_dataloader

task_config = PARAMS.TRAIN.SINGLE_TASK
task_id = task_config.TASK_ID
batch_size = task_config.BATCH_SIZE

train_dataloader = create_dataloader(
    task_id=task_id,
    batch_size=batch_size,
    split='train',
)

In [49]:
for i_batch, sample_batched in enumerate(train_dataloader):
    print(i_batch, sample_batched['image'].size(),
          sample_batched['v_landmarks'].size())

0 torch.Size([16, 1, 256, 256]) torch.Size([16, 13, 256, 256])
1 torch.Size([16, 1, 256, 256]) torch.Size([16, 13, 256, 256])
2 torch.Size([16, 1, 256, 256]) torch.Size([16, 13, 256, 256])
3 torch.Size([16, 1, 256, 256]) torch.Size([16, 13, 256, 256])


KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/samehr/Desktop/cephal/cvmt/.venv/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/samehr/Desktop/cephal/cvmt/.venv/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 54, in fetch
    return self.collate_fn(data)
  File "/home/samehr/Desktop/cephal/cvmt/notebooks/ml/utils.py", line 245, in __call__
    out = self.__supervised_training_collation(batch)
  File "/home/samehr/Desktop/cephal/cvmt/notebooks/ml/utils.py", line 200, in __supervised_training_collation
    batch = [sample for sample in batch if sample[self.label_key] != None]
  File "/home/samehr/Desktop/cephal/cvmt/notebooks/ml/utils.py", line 200, in <listcomp>
    batch = [sample for sample in batch if sample[self.label_key] != None]
KeyError: 'v_landmarks'
